In [ ]:
import sqlite3 as sql3

In [ ]:
# create the connection object
conn = sql3.connect('data/prtr_en.db')

In [ ]:
# you will then get a connection object “conn”, which represents the connection to the database
conn

In [ ]:
# you can create a database cursor, using the connection object’s cursor method
cursor = conn.execute("SELECT id from facilities")
cursor

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame

In [ ]:
# don't forget to close the database connection:
conn.close()